# 从ERA5LAND获取流域土壤含水量数据

本文先考虑和自己的结果分析，也就是径流以及蒸散发的，另一个是土壤含水量做一个更深度的分析。

所以土壤含水量先补充下数据：上一小节导出的shpfile导入到GEE中去计算流域的ERA5LAND的土壤含水量数据。

其中，因为smap的数据就只有最近五年的，所以估计还是得用其他的数据，比如ERA5LAND的数据来分析，这块需要提前准备数据，GEE算一下。

In [ ]:
import os
import sys
from pathlib import Path

sys.path.append(os.path.dirname(Path(os.path.abspath("")).parent.parent))
import definitions
from hydroSPB.data.source.data_camels import Camels
from hydroSPB.app.streamflow_utils import get_json_file, get_trained_model
from hydroSPB.hydroDL.trainer import stat_result

In [ ]:
camels591_file = "../../example/camels/camels_us_mtl_2001_2021_flow_screen.csv"
# if false, run camels_multioutput_preprocess.py in this directory
os.path.isfile(camels591_file)

True

In [ ]:
import pandas as pd

camels591_ids = pd.read_csv(camels591_file)
camels591_ids

,GAGE_ID
0,1013500
1,1022500
2,1030500
3,1031500
4,1047000
...,...
586,14309500
587,14316700
588,14325000
589,14362250


In [ ]:
import definitions

camels_dir = os.path.join(definitions.DATASET_DIR, "camels", "camels_us")
camels = Camels(camels_dir)

Please Check your directory:
ROOT_DIR of the repo:  /home/ouyangwenyu/code/HydroSPB
DATA_SOURCE_DIR of the repo:  /home/ouyangwenyu/data


In [ ]:
import geopandas as gpd

camels_shpfile = camels.data_source_description["CAMELS_BASINS_SHP_FILE"]
camels_shp = gpd.read_file(camels_shpfile)

In [ ]:
camels_shp

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,geometry
0,1013500,0.0,-68.56551,47.01169,2.303988e+09,277.493490,None,None,647993,"MULTIPOLYGON (((-68.35650 46.90311, -68.35612 ..."
1,1022500,0.0,-68.07313,44.79691,6.203873e+08,103.604200,None,None,312624,"POLYGON ((-67.97836 44.61310, -67.97800 44.613..."
2,1030500,0.0,-68.14985,45.80527,3.676155e+09,174.433896,None,None,662248,"MULTIPOLYGON (((-67.83991 45.36614, -67.83955 ..."
3,1031500,0.0,-69.58119,45.23568,7.665447e+08,304.743490,None,None,309614,"MULTIPOLYGON (((-69.33810 45.12317, -69.33800 ..."
4,1047000,0.0,-70.16213,44.98744,9.049562e+08,379.779978,None,None,310157,"POLYGON ((-70.10847 45.21669, -70.10858 45.216..."
...,...,...,...,...,...,...,...,...,...,...
666,14309500,0.0,-123.74216,42.83017,2.263143e+08,737.623894,None,None,117484,"POLYGON ((-123.81322 42.89103, -123.81312 42.8..."
667,14316700,0.0,-122.64677,43.45290,5.880250e+08,947.553082,None,None,176861,"POLYGON ((-122.49936 43.47688, -122.49972 43.4..."
668,14325000,0.0,-124.02763,42.79249,4.449257e+08,649.241611,None,None,166110,"POLYGON ((-124.07751 42.89822, -124.07716 42.8..."
669,14362250,0.0,-123.12535,42.16583,4.387790e+07,939.577778,None,None,44972,"POLYGON ((-123.15128 42.19624, -123.15118 42.1..."


In [ ]:
camels591_shp = camels_shp.loc[
    camels_shp["hru_id"].isin(camels591_ids["GAGE_ID"].values)
]
camels591_shp

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,geometry
0,1013500,0.0,-68.56551,47.01169,2.303988e+09,277.493490,None,None,647993,"MULTIPOLYGON (((-68.35650 46.90311, -68.35612 ..."
1,1022500,0.0,-68.07313,44.79691,6.203873e+08,103.604200,None,None,312624,"POLYGON ((-67.97836 44.61310, -67.97800 44.613..."
2,1030500,0.0,-68.14985,45.80527,3.676155e+09,174.433896,None,None,662248,"MULTIPOLYGON (((-67.83991 45.36614, -67.83955 ..."
3,1031500,0.0,-69.58119,45.23568,7.665447e+08,304.743490,None,None,309614,"MULTIPOLYGON (((-69.33810 45.12317, -69.33800 ..."
4,1047000,0.0,-70.16213,44.98744,9.049562e+08,379.779978,None,None,310157,"POLYGON ((-70.10847 45.21669, -70.10858 45.216..."
...,...,...,...,...,...,...,...,...,...,...
666,14309500,0.0,-123.74216,42.83017,2.263143e+08,737.623894,None,None,117484,"POLYGON ((-123.81322 42.89103, -123.81312 42.8..."
667,14316700,0.0,-122.64677,43.45290,5.880250e+08,947.553082,None,None,176861,"POLYGON ((-122.49936 43.47688, -122.49972 43.4..."
668,14325000,0.0,-124.02763,42.79249,4.449257e+08,649.241611,None,None,166110,"POLYGON ((-124.07751 42.89822, -124.07716 42.8..."
669,14362250,0.0,-123.12535,42.16583,4.387790e+07,939.577778,None,None,44972,"POLYGON ((-123.15128 42.19624, -123.15118 42.1..."


In [ ]:
camels591_dir = "../../example/camels/"
camels591_shp.to_file(os.path.join(camels591_dir, "camels591.shp"))